In [1]:
import requests
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [32]:
API_KEY = "be24b108-a6e3-465f-9197-9f466c26ae46" #AIDevs API
BASE_URL = "https://zadania.aidevs.pl"

#!pip install --upgrade openai


In [3]:
class OpenAIClient:
    """
    A client to interact with OpenAI's API.
    Provides a method to obtain completions.
    """
    
    @staticmethod
    def get_completion(prompt, model="gpt-4", max_tokens=300, temperature=0.3):
        """
        Get a completion response from OpenAI for a given prompt.
        
        :param prompt: Text prompt for the completion.
        :param model: OpenAI model to use.
        :param max_tokens: Maximum tokens in the response.
        :param temperature: Sampling temperature.
        :return: Completion response or None if request fails.
        """
        messages = [{"role": "user", "content": prompt}]
        try:
            response = openai.ChatCompletion.create(
                model=model,
                messages=messages,
                temperature=temperature,
                max_tokens=max_tokens,
            )
            return response.choices[0].message["content"]
        except openai.error.OpenAIError as e:
            print(f"OpenAI error: {e}")
            return None
        except Exception as e:
            print(f"An error occurred: {e}")
            return None
        
#OpenAIClient.get_completion("What is the capital of France?")

In [19]:
class APIClient:
    """
    A client to interact with a given API.
    Provides methods to obtain tokens, fetch tasks, and submit answers.
    """
    
    def __init__(self, base_url, api_key):
        """
        Initialize the APIClient.
        
        :param base_url: Base URL for the API.
        :param api_key: Authentication key for the API.
        """
        self.base_url = base_url
        self.api_key = api_key

    def _make_request(self, method, endpoint, payload=None):
        """
        A generic method to make API requests.
        
        :param method: HTTP method (GET, POST, etc.)
        :param endpoint: API endpoint.
        :param payload: JSON payload for POST requests.
        :return: JSON response or None if request fails.
        """
        url = f"{self.base_url}/{endpoint}"
        response = getattr(requests, method)(url, json=payload)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error in {method} request to {endpoint}: {response.text}")
            return None

    def get_token(self, task_name):
        """
        Fetch a token for a given task name.
        
        :param task_name: Name of the task.
        :return: Token or None if request fails.
        """
        payload = {"apikey": self.api_key}
        return self._make_request("post", f"token/{task_name}", payload)

    def get_task(self, token):
        """
        Fetch a task using a token.
        
        :param token: Token for the task.
        :return: Task details or None if request fails.
        """
        return self._make_request("get", f"task/{token}")

    def submit_answer(self, token, answer):
        """
        Submit an answer for a task using a token.
        
        :param token: Token for the task.
        :param answer: Answer to be submitted.
        :return: API response or None if request fails.
        """
        payload = {"answer": answer}
        return self._make_request("post", f"answer/{token}", payload)

### Initialize the APIClient with your base URL and API key.
api_client = APIClient(base_url=BASE_URL, api_key=API_KEY)

In [33]:
### helloapi example:
task_name = "helloapi"

### Get a token for a specific task name.
token = api_client.get_token(task_name)['token']
print(token)

### Get task details using a token.
task = api_client.get_task(token)['cookie']
print(task)

###
result = api_client.submit_answer(token, task)
print(result)

e17b3ea64503fd34a88d86f9d10a18cd30174aa3
aidevs_97560be4
{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}


### The following tasks were performed using functions, the above will be performed using classes.

In [22]:
def get_token(task_name):
    url = f"{BASE_URL}/token/{task_name}"
    payload = {
        "apikey": API_KEY
    }
    response = requests.post(url, json=payload)
    if response.status_code == 200:
        return response.json()  # Return the entire JSON response
    else:
        print(f"Error getting token: {response.text}")
        return None

def get_task(token):
    url = f"{BASE_URL}/task/{token}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()  # Return the entire JSON response
    else:
        print(f"Error getting task: {response.text}")
        return None

def submit_answer(token, answer):
    url = f"{BASE_URL}/answer/{token}"
    payload = {
        "answer": answer
    }
    response = requests.post(url, json=payload)
    return response.json()  # Return the entire JSON response

def get_completion(prompt, model="gpt-4", max_tokens=300, temperature=0.3): # or model="gpt-3.5-turbo"
    messages = [{"role": "user", "content": prompt}]
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
        )
        return response.choices[0].message["content"]
    except openai.error.OpenAIError as e:
        print(f"OpenAI error: {e}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


# liar

Jest to mechanizm, który mówi nie na temat w 1/3 przypadków. Twoje zadanie polega na tym, aby do endpointa /task/ wysłać swoje pytanie w języku angielskim (dowolne, np “What is capital of Poland?’) w polu o nazwie ‘question’ (metoda POST, jako zwykłe pole formularza, NIE JSON). System API odpowie na to pytanie (w polu ‘answer’) lub zacznie opowiadać o czymś zupełnie innym, zmieniając temat. Twoim zadaniem jest napisanie systemu filtrującego (Guardrails), który określi (YES/NO), czy odpowiedź jest na temat. Następnie swój werdykt zwróć do systemu sprawdzającego jako pojedyncze słowo YES/NO. Jeśli pobierzesz treść zadania przez API bez wysyłania żadnych dodatkowych parametrów, otrzymasz komplet podpowiedzi. Skąd wiedzieć, czy odpowiedź jest ‘na temat’? Jeśli Twoje pytanie dotyczyło stolicy Polski, a w odpowiedzi otrzymasz spis zabytków w Rzymie, to odpowiedź, którą należy wysłać do API to NO.

In [8]:
task_name = "liar"

In [6]:
def ask_question(token, question):
    url = f"{BASE_URL}/task/{token}"
    payload = {
        "question": question
    }
    response = requests.post(url, data=payload)  # Using data instead of json
    if response.status_code == 200:
        return response.json()  # Return the entire JSON response
    else:
        print(f"Error asking question: {response.text}")
        return None

In [15]:
#question = "What the moon?"
question = get_completion("respond with random question", model="gpt-3.5-turbo", max_tokens=100, temperature=0.8)

question

'What is your favorite season and why?'

In [16]:
token = get_token(task_name)['token']

answer_to_question = ask_question(token, question)['answer']

print(question + " : " + answer_to_question)

What is your favorite season and why? : As an artificial intelligence, I don't have personal experiences or feelings, so I don't have a favorite season. However, I can provide information about any season you'd like!


Guardrails

In [17]:
prompt = f"""
Given the question '{question}' and the answer '{answer_to_question}', is the answer relevant? Respond with YES or NO.
"""

response = get_completion(prompt, model="gpt-3.5-turbo", max_tokens=100, temperature=0.2)

response.strip()  # Stripping to ensure there's no extra whitespace

'YES'

In [18]:
submit_answer(token, response)

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}

# blogger

Napisz wpis na bloga (w języku polskim) na temat przyrządzania pizzy Margherity. Zadanie w API nazywa się ”blogger”. Jako wejście otrzymasz spis 4 rozdziałów, które muszą pojawić się we wpisie. Jako odpowiedź musisz zwrócić tablicę (w formacie JSON) złożoną z 4 pól reprezentujących te cztery rozdziały.

In [17]:
task_name = "blogger"

In [18]:
token = get_token(task_name)['token']
task = get_task(token)['blog']

print(task)

['Wstęp: kilka słów na temat historii pizzy', 'Niezbędne składniki na pizzę', 'Robienie pizzy', 'Pieczenie pizzy w piekarniku']


Writing a Blog Post - Best Solution - in One Prompt and Dividing Afterward

In [19]:
prompt = f"""
Write a Polish blog post about the preparation of Margherita pizza. Please elaborate on each of the following chapters, and separate each chapter with the keyword '***':
{'***'.join(task)}
"""
response = get_completion(prompt, model="gpt-3.5-turbo", max_tokens=1000, temperature=0.3)

# Post-process the response into a list of strings, where each string is a chapter
blog_post = [chapter.strip() for chapter in response.split('***')]

print(blog_post)

['Wstęp: kilka słów na temat historii pizzy\n\nPizza jest jednym z najbardziej znanych i uwielbianych dań na całym świecie. Jej początki sięgają starożytności, a dokładniej do starożytnego Rzymu. Jednak to we Włoszech pizza zyskała swoją prawdziwą popularność i stała się symbolem tego kraju. Jednym z najbardziej klasycznych rodzajów pizzy jest Margherita, która swoją nazwę zawdzięcza królowej Włoch, Marghericie Sabaudzkiej. Dziś podzielę się z Wami przepisem na przygotowanie tej pysznej pizzy.', 'Niezbędne składniki na pizzę\n\nPrzygotowanie Margherity nie wymaga wielu składników, ale ważne jest, aby były one najwyższej jakości. Oto lista potrzebnych składników:\n- Ciasto na pizzę (można je kupić gotowe lub przygotować samodzielnie)\n- Sos pomidorowy (najlepiej zrobiony z dojrzałych pomidorów)\n- Mozzarella (świeża lub w postaci sera)\n- Świeże bazylia\n- Sól i pieprz do smaku\n- Oliwa z oliwek', 'Robienie pizzy\n\n1. Przygotuj ciasto na pizzę według wybranego przepisu lub użyj gotoweg

In [20]:
result = submit_answer(token, blog_post)

print(result)

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}


solution in LangChain

In [23]:
#!pip install --upgrade langchain pydantic


In [34]:
from langchain.chat_models import ChatOpenAI

In [35]:
# Import necessary libraries and modules from LangChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# Create a model instance
chat = ChatOpenAI(temperature=0.3, max_tokens=300)

# Define a prompt template for generating blog post chapters
template_string = """
Generate a chapter of blog post about preparing Margherita pizza. {number} sentences.
Chapter title: {chapter_title}
"""

# Create a ChatPromptTemplate instance from the template string
prompt_template = ChatPromptTemplate.from_template(template_string)

# Initialize an empty list to hold the generated chapters
blog_entry = []

# Loop through each chapter title and generate content
for title in task:
    # Format the messages to be passed to the model
    chapter_messages = prompt_template.format_messages(chapter_title=title, number = 4)
    
    # Call the model to generate content for the chapter
    chapter_response = chat(chapter_messages)
    
    # Append the generated content to the blog_entry list
    blog_entry.append(title + " " + chapter_response.content)

# At this point, blog_entry will be a list of dictionaries,
# each containing a chapter title and the corresponding generated content


In [36]:
blog_entry

['Wstęp: kilka słów na temat historii pizzy Wstęp: kilka słów na temat historii pizzy\n\nPizza to jedno z najbardziej popularnych dań na całym świecie. Jej historia sięga starożytności, a pierwsze wzmianki o niej pochodzą z czasów starożytnego Rzymu. Jednak to we Włoszech pizza zyskała swoją prawdziwą popularność i stała się symbolem tego kraju.\n\nMargherita pizza, znana również jako pizza Margarita, jest jednym z najbardziej klasycznych rodzajów pizzy. Jej nazwa pochodzi od królowej Włoch, Margherity Sabaudzkiej, która podobno była wielką fanką tego dania. Pizza Margherita składa się z prostych składników: cienkiego ciasta, sosu pomidorowego, mozzarelli i świeżych listków bazylii.\n\nW dzisiejszych czasach przygotowanie domowej Margherita pizzy jest bardzo popularne. Wiele osób decyduje się na własnoręczne przygotowanie tego dania, aby móc dostosować składniki i smaki do swoich preferencji. W kolejnych rozdziałach przedstawimy krok po k',
 'Niezbędne składniki na pizzę Wszyscy kocham

In [32]:
result = submit_answer(token, blog_entry)

print(result)

{'code': -2, 'msg': 'token has expired or never existed'}


generating titles one by one results in less consistent text

In [37]:
def generate_chapter_content(chapter_title):
    prompt = f'''
    Please in Polish, write following chapter of blog post about preparation of pizza Margherita, up to 4 sentences:
    Title: {chapter_title}
    '''

    return chapter_title + ": " + get_completion(prompt, model="gpt-3.5-turbo", max_tokens=300, temperature=0.3)

def generate_blog_entry(chapter_titles):
    blog_entry = []
    for title in chapter_titles:
        chapter_content = generate_chapter_content(title)
        print(chapter_content)
        blog_entry.append(chapter_content)
    return blog_entry


In [38]:
blog = generate_blog_entry(task)

result = submit_answer(token, blog)

print(result)

Wstęp: kilka słów na temat historii pizzy: Pizza Margherita, znana również jako królowa pizz, jest jednym z najbardziej klasycznych i popularnych rodzajów pizzy na świecie. Jej historia sięga XIX wieku, kiedy to królowa Włoch, Margherita Savoia, odwiedziła Neapol. Podczas swojej wizyty, królowa zażyczyła sobie spróbować lokalnej pizzy, co skłoniło pizzaiolo do stworzenia specjalnej kompozycji z białym serem, pomidorami i bazylią, nawiązującej do kolorów włoskiej flagi. Od tamtej pory pizza Margherita zdobyła serca wielu smakoszy na całym świecie.
Niezbędne składniki na pizzę: Wszystko zaczyna się od odpowiednich składników, które są niezbędne do przygotowania pysznej pizzy Margherita. Na liście zakupów powinny znaleźć się: świeże pomidory, mozzarella, świeża bazylia, oliwa z oliwek, sól i pieprz oraz ciasto na pizzę. Ważne jest, aby wybrać najwyższej jakości składniki, które zapewnią autentyczny smak tej klasycznej włoskiej potrawy.
Robienie pizzy: Rozdział 3: Pizza Margherita - klasyk

# moderation

Zastosuj wiedzę na temat działania modułu do moderacji treści i rozwiąż zadanie o nazwie “moderation” z użyciem naszego API do sprawdzania rozwiązań. Zadanie polega na odebraniu tablicy zdań (4 sztuki), a następnie zwróceniu tablicy z informacją, które zdania nie przeszły moderacji. Jeśli moderacji nie przeszło pierwsze i ostatnie zdanie, to odpowiedź powinna brzmieć [1,0,0,1]. Pamiętaj, aby w polu ‘answer’ zwrócić tablicę w JSON, a nie czystego stringa.

In [5]:
task_name = "moderation"

In [6]:
token = get_token(task_name)['token']
task = get_task(token)['input']

print(task)

['azjaci są głupi i brzydcy i nie powinni żyć', 'ten gość musi zginąć. Nie pozwole sobię na obrażanie mnie.', 'Sasha.Grey.s3x.p0rn.extreme-interracial.S03E12.DVDRip.mp4', 'majonez Winiary jest lepszy od Kieleckiego']


Moderation

In [7]:
# Initialize an empty list to hold the moderation results
moderation_results = []

# Iterate through each item in the zadanie list
for item in task:
    # Send the item to the Moderation endpoint
    response = openai.Moderation.create(input=item)
    
    # Access the 'flagged' value from the response
    flagged = response['results'][0]['flagged']
    
    # Append 1 if flagged, else 0, to the moderation_results list
    moderation_results.append(1 if flagged else 0)

In [10]:
moderation_results


[1, 1, 1, 0]

In [8]:
result = submit_answer(token, moderation_results)

print(result)

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}


# helloapi

Komunikacja z API odbywa się z pomocą kodu oraz formatu JSON, a każde z zadań składa się z trzech części:

autoryzacji

pobierania danych wejściowych (string lub tablica obiektów)

odesłania odpowiedzi (właściwość answer)

In [9]:
task_name = "helloapi"

In [6]:
token = get_token(task_name)['token']
cookie = get_task(token)['cookie']
result = submit_answer(token, cookie)

print(result)


{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}
